In [ ]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

: 

In [ ]:
distance_matrix = [
    # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
    # fmt: on
]

shop_demands = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]

vehicle_capacities = [25, 25, 30, 20, 20, 30]

depot = 0

vehicle_restricted_roads = {
    0: [(3, 4)],  # Vehicle 0 should avoid edge 3 -> 4
    1: [(1, 2), (2, 1)],  # Vehicle 1 should avoid edges 1 -> 2 and 2 -> 1
}

vehicle_restricted_locations = {3: [1]}

restricted_roads = [(3, 4), (4, 3)]

same_route_location = [[1, 6]]

: 

In [46]:
def create_data_model(
    distance_matrix,
    shop_demands,
    vehicle_capacities,
    depot,
    vehicle_restricted_locations=None,
    vehicle_restricted_roads=None,
    same_route_location=None,
):
    data = {}

    data["distance_matrix"] = distance_matrix
    data["shop_demands"] = shop_demands
    data["vehicle_capacities"] = vehicle_capacities
    data["depot"] = depot
    data["vehicle_restricted_locations"] = vehicle_restricted_locations
    data["vehicle_restricted_roads"] = vehicle_restricted_roads
    data["same_route_location"] = same_route_location
    data["num_vehicles"] = len(vehicle_capacities)

    return data

In [47]:
def get_solution(data, manager, routing, solution):

    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")

    total_distance = 0
    total_load = 0

    for vehicle_id in range(data['num_vehicles']):
        index =  routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'

        route_distance = 0
        route_load = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['shop_demands'][node_index]
            plan_output += f' {node_index} Load({route_load}) -> '
            previous_index = index

            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}km\n"
        plan_output += f"Load of the route: {route_load}\n"

        print(plan_output)
        total_distance += route_distance
        total_load += route_load

    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    

In [48]:
# Instantiate the data problem.
data = create_data_model(
    distance_matrix=distance_matrix,
    shop_demands=shop_demands,
    vehicle_capacities=vehicle_capacities,
    depot=depot,
    vehicle_restricted_locations=vehicle_restricted_locations,
)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [49]:
# Define cost of each arc for each vehicle.
def create_vehicle_cost_callback(vehicle_id, data, manager):
    """Returns a transit callback that accounts for vehicle-specific and general restrictions."""

    def transit_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Check if the road is restricted specifically for the given vehicle
        if (from_node, to_node) in data.get("vehicle_restricted_roads", {}).get(
            vehicle_id, []
        ):
            return int(1e6)  # High cost to avoid this edge for this vehicle
        
        # Check if the road is restricted generally for all vehicles
        if (from_node, to_node) in data.get("restricted_roads", []):
            return int(1e6)  # High cost to avoid this edge for all vehicles

        # Default distance from the distance matrix
        return data["distance_matrix"][from_node][to_node]

    return transit_callback

In [50]:
# Register vehicle-specific transit callbacks
for vehicle_id in range(data["num_vehicles"]):
    transit_callback = create_vehicle_cost_callback(vehicle_id, data, manager)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    routing.SetArcCostEvaluatorOfVehicle(transit_callback_index, vehicle_id)

In [51]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]


demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

True

In [52]:
# Add Distance constraint.
dimension_name = "Distance"
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    3000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [53]:
# Define Transportation Requests.
if data["same_route_location"] is not None:
    for request in data["same_route_location"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
else:
    print("Loactions for same route not provided.!")

Loactions for same route not provided.!


In [54]:
# Avoid specific locations for specific vehicles
if data["vehicle_restricted_locations"] is not None:  # {vehicle_id: [location_ids]}
    for vehicle_id, locations in data["vehicle_restricted_locations"].items():
        for location in locations:
            index = manager.NodeToIndex(location)
            routing.solver().Add(routing.VehicleVar(index) != vehicle_id)

else:
    print("Vehicle-specific restricted locations not provided.!")

In [55]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
)

In [56]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [57]:
if solution:
    get_solution(data, manager, routing, solution)

Objective: 0
Route for vehicle 0:
 0 Load(0) ->  16 Load(8) ->  15 Load(16) ->  14 Load(20) ->  13 Load(24) ->  12 Load(26) ->  11 Load(27) ->  10 Load(29) ->  9 Load(30) ->  8 Load(38) ->  7 Load(46) ->  6 Load(50) ->  5 Load(52) ->  4 Load(56) ->  3 Load(58) ->  2 Load(59) ->  1 Load(60) ->  0 Load(60)
Distance of the route: 0km
Load of the route: 60

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0km
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0km
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0km
Load of the route: 0

Route for vehicle 4:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0km
Load of the route: 0

Route for vehicle 5:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0km
Load of the route: 0

Total distance of all routes: 0m
Total load of all routes: 60
